__Частина 1__

В якості домашнього завдання вам пропонується створити нейронну мережу за допомогою механізмів `Keras`, яка буде класифікувати товари із датасету `fasion_mnist`.

На відміну від попереднього завдання вам пропонується створити `згорткову нейромережу`. Підберіть архітектуру мережі та навчіть її на даних із датасету fasion_mnist. Спробуйте досягти `максимально можливої точності класифікації` за рахунок `маніпуляції параметрами` мережі. `Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання`. Зробіть висновки.
<br><br>
__Частина 2__

В цій частині ми знову будемо працювати з датасетом `fasion_mnist`.

На відміну від попереднього завдання вам пропонується створити `згорткову нейромережу`, що використовує `VGG16` в якості згорткової основи.

Навчіть отриману мережу на даних із датасету fasion_mnist. Спробуйте досягти `максимально можливої точності класифікації` за рахунок `маніпуляції параметрами` мережі. Під час навчання використовуйте `прийоми донавчання` та `виділення ознак`.

`Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання`. Зробіть висновки.

In [93]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.activations import mish, relu, sigmoid, softmax
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras import losses
from keras import metrics
from keras import models
from keras import optimizers
from sklearn.model_selection import train_test_split

# __Part 1__

##### __Dataset__

- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data

In [104]:
(images_train, labels_train), (images_test, labels_test) = fashion_mnist.load_data()
images_train.dtype, images_train[0].shape

(dtype('uint8'), (28, 28))

In [105]:
# each sample (28x28) = 784
NUM_FEATURES = images_train[0].shape[0] * images_train[0].shape[1]
NUM_FEATURES

784

In [106]:
images_train, images_test = np.array(images_train, np.float32), np.array(images_test, np.float32)  # .astype('float32')

# images_train, images_test = images_train.reshape([-1, NUM_FEATURES]), images_test.reshape([-1, NUM_FEATURES])
print(min(images_train.min(), images_test.min()))
maximum = max(images_train.max(), images_test.max()) # np.max([images_train.max(), images_test.max()]) 
maximum

0.0


255.0

In [107]:
# Normalize (min = 0)
images_train, images_test = images_train / maximum, images_test / maximum

In [108]:
images_train.shape, images_test.shape, labels_train.shape, labels_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [109]:
max(labels_train.max(), labels_test.max()), min(labels_train.min(), labels_test.min())

(9, 0)

In [110]:
NUM_CLASSES = len(set(np.concatenate((labels_train, labels_test), axis=0)))
NUM_CLASSES

10

In [111]:
images_valid, images_test, labels_valid, labels_test = train_test_split(
                                                                        images_test, 
                                                                        labels_test, 
                                                                        test_size=0.5, 
                                                                        shuffle=True, 
                                                                        stratify=labels_test)  # (stratify для рівномірного розподілу за значенням y)

In [112]:
labels_train = tf.one_hot(labels_train, depth=NUM_CLASSES).numpy()
labels_test = tf.one_hot(labels_test, depth=NUM_CLASSES).numpy()
labels_valid = tf.one_hot(labels_valid, depth=NUM_CLASSES).numpy()
# labels_train = to_categorical(labels_train, num_classes = NUM_CLASSES)
# labels_valid = to_categorical(labels_valid, num_classes = NUM_CLASSES)
labels_train.shape, labels_test.shape, labels_valid.shape  # ((60000, 10), (10000, 10))

((60000, 10), (5000, 10), (5000, 10))

In [113]:
images_train.shape, images_test.shape, images_valid.shape, labels_train.shape, labels_test.shape, labels_valid.shape

((60000, 28, 28),
 (5000, 28, 28),
 (5000, 28, 28),
 (60000, 10),
 (5000, 10),
 (5000, 10))

##### __Model__

- https://keras.io/api/layers/convolution_layers/convolution2d/
- https://keras.io/api/layers/pooling_layers/max_pooling2d/
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten

In [114]:
kernel_window_size_1 = 3  # 5, 9, 11
kernel_window_size_2 = 3  # 5, 9, 11
kernel_window_size_3 = 3  # 5, 9, 11
filters_1 = 32  # output_depth
filters_2 = 64
filters_3 = 64
filters_4 = 64
# activation_functions = ['relu' for _ in range(4)] + ['softmax']

In [115]:
model = models.Sequential([
                           layers.Conv2D(
                                         filters_1, 
                                         (kernel_window_size_1, kernel_window_size_1), 
                                         activation='relu', 
                                         input_shape=(images_train.shape[1], images_train.shape[2], 1)  # channels 1 - gray, 3 - RGB
                                         ),  
                           layers.MaxPooling2D((2, 2)),

                           layers.Conv2D(
                                         filters_2, 
                                         (kernel_window_size_2, kernel_window_size_2), 
                                         activation='relu'
                                         ),
                           layers.MaxPooling2D((2, 2)),

                           layers.Conv2D(
                                         filters_3, 
                                         (kernel_window_size_3, kernel_window_size_3), 
                                         activation='relu'
                                         ),

                           layers.Flatten(),
                           layers.Dense(
                                        filters_4, 
                                        activation='relu'
                                        ),

                           layers.Dense(
                                        NUM_CLASSES, 
                                        activation='softmax'
                                        )
                           ])

In [116]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 3, 3, 64)          36928     
                                                                 
 flatten_4 (Flatten)         (None, 576)              

In [117]:
model.compile(
              optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

##### __Training__

In [118]:
history = model.fit(images_train, labels_train, epochs=5, batch_size=64)  # w/o valid data

Epoch 1/5
938/938 [==============================] - 9s 9ms/step - loss: 0.5649 - accuracy: 0.7914
Epoch 2/5
938/938 [==============================] - 9s 10ms/step - loss: 0.3370 - accuracy: 0.8785
Epoch 3/5
938/938 [==============================] - 9s 10ms/step - loss: 0.2838 - accuracy: 0.8967
Epoch 4/5
938/938 [==============================] - 10s 10ms/step - loss: 0.2527 - accuracy: 0.9071
Epoch 5/5
938/938 [==============================] - 10s 10ms/step - loss: 0.2282 - accuracy: 0.9164


##### __Error Analysis__

In [119]:
history.history

{'loss': [0.5649124383926392,
  0.33701926469802856,
  0.28380846977233887,
  0.2527290880680084,
  0.22820459306240082],
 'accuracy': [0.7913500070571899,
  0.8785333037376404,
  0.8966666460037231,
  0.9071000218391418,
  0.9164333343505859]}

In [120]:
valid_loss, valid_acc = model.evaluate(images_valid, labels_valid)
print(valid_acc)
test_loss, test_acc = model.evaluate(images_test, labels_test)
print(test_acc)

157/157 [==============================] - 1s 3ms/step - loss: 0.2791 - accuracy: 0.9038
0.9038000106811523
157/157 [==============================] - 0s 3ms/step - loss: 0.2779 - accuracy: 0.9006
0.900600016117096


In [121]:
def draw_2twin_2d(y00: np.array, y01: np.array, y10: np.array, y11: np.array, suptitle: str) -> None:
    x = range(1, len(y00) + 1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 8))

    axs[0].plot(x, y00, 'bo', label='Training loss')
    axs[0].plot(x, y01, 'g', label='Validation loss')
    axs[0].title.set_text('Training and validation loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(x, y10, 'bo', label='Training acc')
    axs[1].plot(x, y11, 'g', label='Validation acc')
    axs[1].title.set_text('Training and validation acc')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Acc')
    axs[1].legend()
    axs[1].grid()
    
    fig.suptitle(suptitle)
    # plt.legend()
    # plt.grid()
    axs[1].grid()
    plt.show()

##### __repeating+__

In [ ]:
def create_NN(n_hidden_: list, activation: list, optimizer=optimizers.SGD, learning_rate=0.1) -> Sequential:
    model = Sequential(name=f'''m-{''.join(map(str, [f'{i}-' for i in n_hidden_[1:-1]]))}{''.join(map(str, [f'{a}-' for a in activation[:-1]]))}{optimizer.__name__}-{learning_rate}''')
    n_layers = len(n_hidden_)
    for layer in range(1, n_layers):
        model.add(Dense(n_hidden_[layer], activation=activation[layer-1], input_shape=(n_hidden_[layer-1],)))

    model.compile(
                  optimizer=optimizer(learning_rate=learning_rate), # legacy
                  loss=losses.CategoricalCrossentropy(),  # 'categorical_crossentropy' losses.binary_crossentropy,
                  metrics=['accuracy', metrics.CategoricalAccuracy()]  # metrics.BinaryAccuracy()  CategoricalAccuracy()
                  )

    return model

In [ ]:
models = []  # 9216 models # !!! more then 32 GB RAM!!!!, ... try 1260 = 28sec
for set_layers in n_hiddens_:  # 584
    for act in bundle_activations:  # 39
        if len(set_layers) == (len(act) + 1):  # 1024  -> 210
            for opt in optimizators:  # 3
                for lr in learning_rates:  # 2
                    models.append(create_NN(n_hidden_=set_layers, activation=act, optimizer=opt, learning_rate=lr))

In [ ]:
def retest_NN(set_layers: list, act: list, opt: optimizers, lr: float, bs: int, epochs: int=100) -> None:
    """For retest one model."""
    models = [create_NN(
                        n_hidden_=set_layers, 
                        activation=act, 
                        optimizer=opt, 
                        learning_rate=lr
                        )] 

    history = {}
    history[f'{models[0].name}({bs})'] = models[0].fit(
                                                       x_train,
                                                       y_true,
                                                       epochs=epochs,
                                                       batch_size=bs,
                                                       validation_data=(x_test, y_truev)
                                                       )

    for model in history:
        history_dict = history[model].history

        loss_values = history_dict['loss']
        val_loss_values = history_dict['val_loss']
        acc_values = history_dict['categorical_accuracy']
        val_acc_values = history_dict['val_categorical_accuracy']   

        draw_2twin_2d(loss_values, val_loss_values, acc_values, val_acc_values, suptitle=model)

##### __Сonclusions 1__

# __Part 2__

##### __Dataset__

##### __Model__

##### __Training__

##### __Error Analysis__

##### __Сonclusions 2__